In [ ]:
# Import the pandas library for reading and manipulating your data
# Anywhere you see "pd" in this notebook it's a reference to the pandas library
import pandas as pd
# Extra step to ensure that pandas plays nice with matplotlib
pd.plotting.register_matplotlib_converters()
# Import the numpy library for running calculations on your data
# Anywhere you see "np" in this notebook it's a reference to the numpy library
import numpy as np
# Import some components of the matplotlib library for plotting your data
# Anywhere you see "plt" or "mpimg" in this notebook it's a reference to the "pyplot" and "image" packages from matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Import seaborn library for making your plots pretty!
# Anywhere you see "sns" in this notebook it's a reference to the seaborn library
import seaborn as sns
# Set some default plotting parameters using seaborn
sns.set()

In [ ]:
# Store file name as string
csv_string_2016 = "Parking_Violations_Issued_-_Fiscal_Year_2016.csv"
csv_string_2017 = "Parking_Violations_Issued_-_Fiscal_Year_2017.csv"

# Convert full dataset csvs to dataframes
df_16 = pd.read_csv(csv_string_2016)
df_17 = pd.read_csv(csv_string_2017)



In [ ]:
# Concatenate the 2016 and 2017 data
df = pd.concat([df_16, df_17], ignore_index=True)

In [ ]:
# Identify columns from the dataset that we are interested in keeping
cols_to_keep = ['Plate ID',
                'Registration State',
                'Plate Type',
                'Issue Date',
                'Violation Code',
                'Vehicle Body Type',
                'Vehicle Make',
                'Vehicle Color',
                'Violation Time',
                'Violation County',
                'Vehicle Year']
# Remove the unnecessary columns
cols_to_remove = [col for col in df.columns if col not in cols_to_keep]
df = df.drop(columns = cols_to_remove)

# Rename the columns to have underscores instead of spaces for easier access
cols_to_keep_rename = {}
for col in cols_to_keep:
    renamed = col.replace(" ", "_").lower()
    cols_to_keep_rename[col] = renamed
df = df.rename(columns = cols_to_keep_rename)

In [ ]:
# Filter data by matching values in each column

# Store the codes for color, county, body, make, and violation to ease data cleaning
color_codes = ['BK', 'BL', 'BR', 'GL', 'GY',
               'MR', 'OR', 'PK', 'PR', 'RD',
               'TN', 'WH', 'YW','BLACK', 'BLUE', 
               'GREY', 'GREEN', 'RED', 
               'WHITE', 'YELLOW', 'SILVE', 'BROWN', 'GOLD', 'PINK']

county_codes = ["BX", "K", "NY", "Q", "R"]

body_codes_to_keep = ['CONV', 'SUBN', '4DSD',
              '2DSD', 'PICK', 'VAN']

vehicle_makes_to_keep  = ['ACURA', 'AUDI', 'BMW', 'BUICK', 'CADIL',
                          'CHEVR', 'CHRYS', 'DODGE', 'FIAT', 'FORD',
                          'GMC', 'HONDA', 'HUMME', 'HYUND', 'INFIN',
                          'ISUZU', 'JAGUA', 'JEEP', 'KIA', 'LEXUS',
                          'LINCO', 'MASSA', 'MAZDA', 'ME/BE', 'MERCU',
                          'MINI', 'MITSU', 'NISSA', 'OLDSM', 'PLYMO',
                          'PONTI', 'PORSC', 'ROVER', 'SAAB', 'SATUR',
                          'SMART', 'SUBAR', 'SUZUK', 'TESLA', 'TOYOT',
                          'VOLKS', 'VOLVO']

violation_codes_to_keep = [20, 21, 37, 38, 40]

# Apply filters
print(f'{len(df)} rows before filtering')
# Clean data by filtering out rows that do not conform to lists above
df = df[df.vehicle_color.isin(color_codes)]
print(f'{len(df)} rows after color filtering')
df = df[df.vehicle_body_type.isin(body_codes_to_keep)]
print(f'{len(df)} rows after body type filtering')
df = df[df.violation_code.isin(violation_codes_to_keep)]
print(f'{len(df)} rows after violation code filtering')
df = df[df.vehicle_make.isin(vehicle_makes_to_keep)]
print(f'{len(df)} rows after vehicle make filtering')
df = df[df.violation_county.isin(county_codes)]
print(f'{len(df)} rows after county code filtering')
df = df[df.plate_type == 'PAS']
print(f'{len(df)} rows after plate type filtering')
df = df[df.vehicle_year != 0]
df = df[df.vehicle_year <= 2017]
print(f'{len(df)} rows after vehicle year filtering')
df = df[df.issue_date.astype(str).str.contains("2016")]
print(f'{len(df)} rows after restricting to 2016')


In [ ]:
# Convert time to proper format for to_datetime
df['violation_time'] = df['violation_time'].apply(\
             lambda x: "12"+x[2:]+"M" if x[:2] == "00" else x+"M")
# Combine columns into new timestamp column
df['timestamp'] = (df.issue_date.astype(str) + \
                            " " + df.violation_time.astype(str))

# Convert timestamp col into datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], \
                             errors = 'coerce', format = "%m/%d/%Y %I%M%p")

# Drop NaT rows that were unable to convert to datetime
df.dropna(inplace=True)
# Drop any duplicate rows
df.drop_duplicates(keep=False,inplace=True)
# Sort by timestamp
df.sort_values(by=['timestamp'], inplace=True)
# Set timestamp as index to get rid of goofy index values
df = df.set_index('timestamp')


In [ ]:
# Create color map for values to change and values to keep the same (annoyingly have to map all)
color_map = {'BK':'BLACK', 'BL':'BLUE', 'BR':'BROWN', 'GL':'GOLD', 'GY':'GRAY',
            'MR':'MAROON', 'OR':'ORANGE', 'PK':'PINK', 'PR':'PURPLE', 'RD':'RED',
            'TN':'TAN', 'WH':'WHITE', 'YW':'YELLOW',
            'GREY':'GRAY', 'SILVE':'SILVER', 'BLACK':'BLACK', 'BLUE':'BLUE', 'BROWN':'BROWN', 
            'GOLD':'GOLD', 'GRAY':'GRAY',
            'MAROON':'MAROON', 'ORANGE':'ORANGE', 'PINK':'PINK', 'PURPLE':'PURPLE', 'RED':'RED',
            'TAN':'TAN', 'WHITE':'WHITE', 'YELLOW':'YELLOW',
            'GRAY':'GRAY', 'SILVER':'SILVER'}
df.vehicle_color = df.vehicle_color.map(color_map)
# Map county and violation code as well
county_map = {'K':'Brooklyn', 'Q':'Queens', 'NY':'Manhattan', 'BX':'Bronx', 'R':'Staten Island'}
df.violation_county = df.violation_county.map(county_map)
violation_codes_map = {20:'NO PARKING ZONE', 21:'STREET CLEANING', 37:'NO METER RECEIPT', 
                       38:'EXPIRED METER', 40:'FIRE HYDRANT'}
df.violation_code = df.violation_code.map(violation_codes_map)

In [ ]:
# Verify that things appear to have worked as intended, i.e. sorting, mapping and timestamp conversion
df.head(100)

In [ ]:
df.tail(100)

In [ ]:
# If timestamp conversion appears successful, drop the unneeded violation_time and issue_date columns
# (also drop plate type as no longer needed)
df.drop(columns = ['violation_time', 'issue_date', 'plate_type'], inplace=True)
# Rename county to borough
df = df.rename(columns = {'violation_county':'borough'})
# Convert vehicle year to int
df.vehicle_year = df.vehicle_year.astype(int)

In [ ]:
# Save final df to a new CSV
df.to_csv('cleaned_and_filtered_data_full.csv')

In [ ]:
df.head()